# 3. Convolutional Neural Network models

## 3.2. CNN classifiers for identifying handwritten digits.

#### Load MNIST

In [3]:
from keras.datasets import mnist


Using TensorFlow backend.


In [4]:
def load_mnist():
    return mnist.load_data()

#### MNIST with pure CNN

In [7]:
from autokeras import CnnModule
from autokeras.backend.torch.loss_function import classification_loss
from autokeras.nn.metric import Accuracy
from autokeras.preprocessor import OneHotEncoder
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor

def to_one_hot(y):
    y_encoder = OneHotEncoder()
    y_encoder.fit(y)
    y = y_encoder.transform(y)
    return y, y_encoder


(x_train, y_train), (x_test, y_test) = load_mnist()
y_train, y_encoder = to_one_hot(y_train)
y_test, _ = to_one_hot(y_test)
train_dataset = TensorDataset(Tensor(x_train), Tensor(y_train))
test_dataset = TensorDataset(Tensor(x_test), Tensor(y_test))
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=64)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=64)

cnnModule = CnnModule(loss=classification_loss, metric=Accuracy, verbose=True)
cnnModule.fit(n_output_node=y_encoder.n_classes,
             input_shape=(-1, x_train.shape[1], x_train.shape[2]),
             train_data=train_loader,
             test_data=test_loader,
             time_limit=20*60)


Saving Directory: /tmp/autokeras_SHD8OK

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
Epoch-1, Current Metric - 0:  15%|████                       | 140/938 [00:00<00:05, 156.65 batch/s]

KeyboardInterrupt: 

#### MNIST with ImageClassifier using CNN

In [ ]:
from autokeras.image.image_supervised import ImageClassifier

(x_train, y_train), (x_test, y_test) = load_mnist()

cnnModule = CnnModule(loss=classification_loss, metric=Accuracy, verbose=True)
imageClf = ImageClassifier(verbose=True)
imageClf.cnn = cnnModule
imageClf.fit(x_train, y_train, time_limit=20 * 60)

## 3.3. CNN classifier for identifying other objects.

#### CIFAR100

In [ ]:
# Dataset of 50,000 32x32 color training images => 32x32x3, 
# labeled over 100 categories, and 10,000 test images.

%matplotlib inline
import matplotlib.pyplot as plt
from keras.datasets import cifar100

def to_one_hot(y):
    y_encoder = OneHotEncoder()
    y_encoder.fit(y)
    y = y_encoder.transform(y)
    return y, y_encoder

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

_, canvas = plt.subplots(1, 2)
_ = canvas[0].imshow(x_train[1234])
_ = canvas[1].imshow(x_test[1234])

In [ ]:
y_train, y_encoder = to_one_hot(y_train)
y_test, _ = to_one_hot(y_test)
train_dataset = TensorDataset(Tensor(x_train), Tensor(y_train))
test_dataset = TensorDataset(Tensor(x_test), Tensor(y_test))
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=128)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=128)

cnnModule = CnnModule(loss=classification_loss, metric=Accuracy, verbose=True)
cnnModule.fit(n_output_node=y_encoder.n_classes,
             input_shape=(-1, x_train.shape[3], x_train.shape[1], x_train.shape[2]),
             train_data=train_loader,
             test_data=test_loader,
             time_limit=20*60)

#### Fashion MNIST

In [ ]:
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
y_train, y_encoder = to_one_hot(y_train)
y_test, _ = to_one_hot(y_test)
train_dataset = TensorDataset(Tensor(x_train), Tensor(y_train))
test_dataset = TensorDataset(Tensor(x_test), Tensor(y_test))
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=128)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=128)

cnnModule = CnnModule(loss=classification_loss, metric=Accuracy, verbose=True)
cnnModule.fit(n_output_node=y_encoder.n_classes,
             input_shape=(-1, x_train.shape[1], x_train.shape[2]),
             train_data=train_loader,
             test_data=test_loader,
             time_limit=20*60)

## 3.4. CNN Regressor for MNIST.

In [7]:
from autokeras import CnnModule
from autokeras.backend.torch.loss_function import regression_loss
from autokeras.nn.metric import MSE
# [sum (y_true - y_pred)**2] / num_instances
# MSE = ~9

from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor


(x_train, y_train), (x_test, y_test) = load_mnist()
train_dataset = TensorDataset(Tensor(x_train), Tensor(y_train))
test_dataset = TensorDataset(Tensor(x_test), Tensor(y_test))
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=64)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=64)

cnnModule = CnnModule(loss=regression_loss, 
                      metric=MSE, 
                      verbose=True)
cnnModule.fit(n_output_node=1,
             input_shape=(-1, x_train.shape[1], x_train.shape[2]),
             train_data=train_loader,
             test_data=test_loader,
             time_limit=20*60)


Saving Directory: /tmp/autokeras_I5SA6D

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   1321.952299785614    |    8.3296049118042     |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+
                                                                                

## 3.5. Visualizing the best CNN.

In [8]:
import os
from graphviz import Digraph
# pip install graphviz
from autokeras.utils import pickle_from_file

from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
y_train, y_encoder = to_one_hot(y_train)
y_test, _ = to_one_hot(y_test)
train_dataset = TensorDataset(Tensor(x_train), Tensor(y_train))
test_dataset = TensorDataset(Tensor(x_test), Tensor(y_test))
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=128)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=128)

cnnModule = CnnModule(path='temp', 
                      loss=classification_loss, 
                      metric=Accuracy, 
                      verbose=True)
cnnModule.fit(n_output_node=y_encoder.n_classes,
             input_shape=(-1, x_train.shape[1], x_train.shape[2]),
             train_data=train_loader,
             test_data=test_loader,
             time_limit=20*60)

def to_pdf(graph, path):
    dot = Digraph(comment='Fashion MNIST')

    for index, node in enumerate(graph.node_list):
        dot.node(str(index), str(node.shape))

    for u in range(graph.n_nodes):
        for v, layer_id in graph.adj_list[u]:
            dot.edge(str(u), str(v), str(graph.layer_list[layer_id]))

    dot.render(path)


def visualize(path):
    module = pickle_from_file(os.path.join(path, 'module'))
    module.searcher.path = path
    for item in module.searcher.history:
        model_id = item['model_id']
        graph = module.searcher.load_model_by_id(model_id)
        to_pdf(graph, os.path.join(path, str(model_id)))
 

visualize('temp')

Saving Directory: temp

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   26.461857187747956   |   0.8796799999999999   |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+
                                                                                                 